# Readme

This is a baseline model, which only uses SBERT-based semantic search without TFIDF or any preprocessing

# Preparation

In [1]:
from sentence_transformers import SentenceTransformer
import scipy
import os
import pandas as pd
import numpy as np
from stemming.porter2 import stem
import re
import csv

# preprocessing method
def string_tokenise(string):  # return list
    result = re.findall(r"\w+", string)
    return result


def case_fold(list1):  # return list
    result = [word.lower() for word in list1]
    #     string = ' '.join([str(elem) for elem in list1])
    #     result = string.lower().split() #lower() is the same as casefold()
    return result


def stopping(list1):  # return list
    stopfile = open("englishST.txt", 'r')
    stopwords = stopfile.read().split()
    result = [items for items in list1 if items not in stopwords]
    return result


def normalise(list1):  # return list
    result = []
    for item in list1:
        result.append(stem(item))
    return result

# LOAD BERT SENTENCE MODEL
# Load the BERT model. Various models trained on Natural Language Inference (NLI) https://github.com/UKPLab/sentence-transformers/blob/master/docs/pretrained-models/nli-models.md and 
# Semantic Textual Similarity are available https://github.com/UKPLab/sentence-transformers/blob/master/docs/pretrained-models/sts-models.md

model = SentenceTransformer('bert-base-nli-mean-tokens')

# try different encoding and seperate identifiers
def read_csv(filepath):
     if os.path.splitext(filepath)[1] != '.csv':
          return  # or whatever
     seps = [',', ';', '\t']                    # ',' is default
     encodings = [None, 'utf-8', 'ISO-8859-1', 'utf-16','ascii']  # None is default
     for sep in seps:
         for encoding in encodings:
              try:
                  return pd.read_csv(filepath, encoding=encoding, sep=sep)
              except Exception:  # should really be more specific 
                  pass
     raise ValueError("{!r} is has no encoding in {} or seperator in {}"
                      .format(filepath, encodings, seps))

# Set up (and preprocess) and Embedding

In [2]:
# parse the verified claims
vclaims_directory = 'v3.0/verified_claims.docs.tsv'
vclaims_fields = ['vclaim_id', 'vclaim', 'title']
df_vc = pd.read_csv(vclaims_directory, usecols = vclaims_fields, sep = '\t')
vclaims_list = df_vc.vclaim.tolist().copy()
vclaims_list

# prepare queries and put them in a list
tweets_directory = 'v3.0/train/tweets.queries.tsv'
tweets_fields = ['tweet_id', 'tweet_content']
df_t = pd.read_csv(tweets_directory, usecols = tweets_fields, sep = '\t')
tweets_list = df_t.tweet_content.tolist().copy()
tweets_list



['Trump needs to immediately divest from his businesses and comply with the emoluments clause. Iran could threaten Trump hotels *worldwide* and he could provoke war over the loss of revenue from skittish guests.  His business interests should not be driving military decisions. — Ilhan Omar (@IlhanMN) January 6, 2020',
 'A number of fraudulent text messages informing individuals they have been selected for a military draft have circulated throughout the country this week.',
 'Fact check: The U.S. Army is NOT contacting anyone regarding the draft. If you are receiving texts, phone calls or direct messages about a military draft, they are not official communications from the U.S. Army pic.twitter.com/3S32De8ekP — U.S. Army CGSC (@USACGSC) January 8, 2020',
 'The US drone attack on #Soleimani caught on camera.#IranUsapic.twitter.com/TvRkHvlgby — Olaudah Equiano® (@RealOlaudah) January 6, 2020',
 '1. To the dim witted reporters like @dmedin11:  no one said this wasn’t photoshopped.  No one 

In [19]:
# Use SBERT to do sentence embedding
# Each vclaim is encoded as a 1-D vector with 78 columns
vclaim_embeddings = model.encode(vclaims_list)

# print('Sample BERT embedding vector - length', len(vclaim_embeddings[0]))

np.save('01_embeddings.npy', vclaim_embeddings)
print('Sample BERT embedding vector - note includes negative values', vclaim_embeddings[0])

Sample BERT embedding vector - note includes negative values [-0.06839999  0.11646768 -0.37378955  0.27000183  0.37912527 -1.2374749
  0.5467199  -0.64416355  0.01363655 -0.06171849 -0.23101607 -0.4370634
 -0.11530093  0.78729784 -0.53884095  0.30604655 -0.654617   -0.11056169
 -0.00810371 -0.20159982  0.39454025 -0.6258818  -0.50732946 -0.26715767
  0.86845887  0.3733293  -0.40977922 -0.76450676 -1.4386698   0.29790646
 -0.49806467  0.2177606  -0.03685323 -0.16548534 -0.138993    0.7363141
 -1.177447   -0.17506939 -0.21862906 -0.26186585 -0.07425258 -0.25737232
 -0.6412997  -0.24607065 -0.792142   -0.5476829   0.7522482   0.46411932
  0.578255   -0.30182254  0.29880628 -0.02058476 -0.3748311  -0.7059333
  0.07826146 -0.19672419  0.09062672 -1.7489096  -0.27156785 -0.5553281
 -0.6658741   0.36642843  0.52788115  0.60650444  0.26296413 -0.00902202
  0.4893662   0.8471768  -1.2924407  -1.1877214   0.49134386 -0.5718257
 -0.7279378  -0.45531315  0.3747597  -0.25155854 -0.16524307 -0.14369

# PERFORM SEMANTIC SEARCH AND GENERATE RESULTS

In [5]:
# prepare embeddings
copy = np.load('01_embeddings.npy')
vclaim_embeddings = copy.copy() # duplicate embedding data
tweet_embeddings = model.encode(tweets_list)

# prepare submitted file
RankedIROutput = open('01_results.tsv', 'w')
results_fields = ['tweet_id','Q0','vclaim_id','rank','score','tag']
writer = csv.DictWriter(RankedIROutput, fieldnames = results_fields)
writer.writeheader()

# Find the closest 3 sentences of the corpus for each tweet sentence based on cosine similarity
number_top_matches = 1 #@param {type: "number"}

print("Semantic Search Results")

for tweet, tweet_embedding in zip(tweets_list, tweet_embeddings):
    return_data = [] # a dict that contains the info that needed to be reported
                    # in eval. the structure is {'attr1': value1, 'attr2': value2}
    distances = scipy.spatial.distance.cdist([tweet_embedding], vclaim_embeddings, "cosine")[0]

    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])

    print("\n\n======================\n\n")
    print("tweet:", tweets_list[tweets_list.index(tweet)]) # for best reading, output orginial text reather than tokens
    print("\nTop 20 most similar vclaim in corpus:")

    rank = 0 # initialize param rank in return_data
    for idx, distance in results[0:number_top_matches]:
        print(vclaims_list[idx].strip(), "(Cosine Score: %.4f)" % (1-distance))
        score = "%.4f" % (1-distance) # an parameter that need to be reported in eval
        
        # write the results to the file
        tweet_id = df_t.loc[df_t['tweet_content'] == tweet,'tweet_id'].item()
        vclaim_id = df_vc.loc[df_vc['vclaim'] == vclaims_list[idx],'vclaim_id'].item()
        rank = rank + 1
        score = score
        tag = 'DC'

        return_data = {'tweet_id':tweet_id, 'Q0':'Q0', 'vclaim_id':vclaim_id, 'rank': '1',
                      'score': score, 'tag': tag}
        # NB: no matter what the real rank is, it should be 1 if you want it to be counted
    
        # write data to the results file
        # results_fields = ['tweet_id','Q0','vclaim_id','rank','score','tag']
        # NO HEADER NEEDED!!
        writer = csv.DictWriter(RankedIROutput, fieldnames = results_fields, delimiter='\t')
        writer.writerow(return_data)
            
RankedIROutput.close()

Semantic Search Results




tweet: Trump needs to immediately divest from his businesses and comply with the emoluments clause. Iran could threaten Trump hotels *worldwide* and he could provoke war over the loss of revenue from skittish guests.  His business interests should not be driving military decisions. — Ilhan Omar (@IlhanMN) January 6, 2020

Top 20 most similar vclaim in corpus:
Donald Trump said that America's economy crashing would lead to riots demanding we make the country great again. (Cosine Score: 0.8107)




tweet: A number of fraudulent text messages informing individuals they have been selected for a military draft have circulated throughout the country this week.

Top 20 most similar vclaim in corpus:
Hillary Clinton's campaign made several payments to a company that destroys sensitive data due to an ongoing investigation into her use of e-mail while she was secretary of state. (Cosine Score: 0.7438)




tweet: Fact check: The U.S. Army is NOT contacting anyone regar





tweet: Holy shit Michael Knowles of Daily Wire just called Greta Thunberg a “mentally ill Swedish child.”@ChristopherHahn tore into him pic.twitter.com/Ki0cK6W3Ev — jordan (@JordanUhl) September 23, 2019

Top 20 most similar vclaim in corpus:
A Fox News guest disparaged teenage climate activist Greta Thunberg by calling her "mentally ill." (Cosine Score: 0.7877)




tweet: Trump visited the US border this week & said rock climbers tested his wall & agreed it cannot be climbed. But Mexicans have turned Trump’s wall into a tourist attraction & are playing a game to see who can climb it the fastest, no ladders or ropes needed. The record is 45 seconds pic.twitter.com/2Xr4rolKk2 — Stone 🥶 (@stonecold2050) September 22, 2019

Top 20 most similar vclaim in corpus:
Images shared widely in September 2019 showed people easily climbing the same fence, or same type of fence, that U.S. President Donald Trump said "can't be climbed." (Cosine Score: 0.7574)




tweet: America has more governors 





tweet: The story by Axios that President Trump wanted to blow up large hurricanes with nuclear weapons prior to reaching shore is ridiculous. I never said this. Just more FAKE NEWS! — Donald J. Trump (@realDonaldTrump) August 26, 2019

Top 20 most similar vclaim in corpus:
Donald Trump suggested he'd use nuclear weapons against Mexico if they refused to build and pay for a wall. (Cosine Score: 0.8411)




tweet: .@AllenFrancesMD, a psychiatrist, tells @Brianstelter: Trump “may be responsible for many more million deaths” than Adolf Hitler, Joseph Stalin, and Mao Zedong.https://t.co/cBUZVQjhNUpic.twitter.com/RVARyeVnVh — Julio Rosas (@Julio_Rosas11) August 25, 2019

Top 20 most similar vclaim in corpus:
A psychiatrist said U.S. President Donald Trump may be responsible for millions more deaths than Hitler, Stalin, and Mao. (Cosine Score: 0.8546)




tweet: fox news is very worried about homeless people laying “in the street having just shot up with marijuana” people forget that weed





tweet: Crazy Bernie Sanders recently equated the City of Baltimore to a THIRD WORLD COUNTRY! Based on that statement, I assume that Bernie must now be labeled a Racist, just as a Republican would if he used that term and standard! The fact is, Baltimore can be brought back, maybe…… — Donald J. Trump (@realDonaldTrump) July 29, 2019

Top 20 most similar vclaim in corpus:
President Donald Trump was using an anti-Semitic slur when he called NBC host Chuck Todd 'sleepy eyes.' (Cosine Score: 0.7881)




tweet: GOP rallies around Trump 9/11 role. “While Obama and Biden were cowering in fear on Air Force 1, Mr. Trump was on the ground with first responders searching for survivors and pulling people to safety,” Jim Jordan says. “I remember seeing him on TV, running toward the danger.” pic.twitter.com/P7ol4wycoF — Dan Lyons (@realdanlyons) July 29, 2019

Top 20 most similar vclaim in corpus:
A video shows an audience member scaring Donald Trump at a rally by screaming 'Allahu Akbar.' (Cosin





tweet: WATCH: Elderly Man Beaten With Crowbar, Another Has Head Split Open By Antifa While Trying To Help ‘Gay Man In Sundress’ https://t.co/slRy6Nzajm — The Daily Wire (@realDailyWire) July 1, 2019

Top 20 most similar vclaim in corpus:
Notorious homophobe and Westboro Baptist Church founder Fred Phelps made a deathbed confession that he was gay. (Cosine Score: 0.7459)




tweet: John Blum, the older man that was bloodied and had his picture go viral, is here in this video before the brawl broke out holding his baton. A black man was able to deescalate a stand off between John and #antifa & other counter protesters. #Milkshake#NoPrideforProudBoyspic.twitter.com/yomI2759Bp — John (@Johnnthelefty) July 2, 2019

Top 20 most similar vclaim in corpus:
A photograph of a 'My Daddy's Life Matters' sign included text explaining why its OK for police officers to shoot black men. (Cosine Score: 0.7522)




tweet: In order to get elected, @BarackObama will start a war with Iran. — Donald J. T

...





tweet: Criminalizing flag-burning: Totally criminal when by proposed by Trump (2016) yet somehow totally cool when proposed by Hillary (2005). — John Schindler (@20committee) November 29, 2016

Top 20 most similar vclaim in corpus:
New York Times columnist hailed the election of Donald Trump as a 'complete repudiation of Barack Obama.' (Cosine Score: 0.7918)




tweet: Delivering supplies to #standingrock protest camp could warrant $1,000 fine #Humanrights #INDIGENOUS #environment https://t.co/nSFFliOP7R pic.twitter.com/wPBRz8932S — Dr. Frank Wilson (@DrFrankWilson) November 30, 2016

Top 20 most similar vclaim in corpus:
Anyone bringing supplies to DAPL protesters at the Standing Rock Indian Reservation could be fined $1,000 for the attempt, according to the Morton County Sheriff's Department. (Cosine Score: 0.7774)




tweet: The Sioux are literally being forced at gunpoint to accept ecological risks that North Dakota’s white residents refused. — terrycloth warrior (@suntzufuntzu





tweet: Lmfao soooooo did those Obama phones really get cut off?? This kid just posted his phone stopped working & he still had 500mins left?☠️? — BLM✨ (@blmmxo_) November 9, 2016

Top 20 most similar vclaim in corpus:
A Facebook page is giving away hundreds of PlayStation 4 or PlayStation 5 consoles because they have been unsealed and cannot be sold. (Cosine Score: 0.7776)




tweet: eric trump deleted his violation of new york state law, so here it is for posterity pic.twitter.com/Y0ss1Ew5Wc — noah kulwin (@nkulw) November 8, 2016

Top 20 most similar vclaim in corpus:
After the 2019 El Paso shooting, President Trump deleted tweets that referred to immigrants as 'invaders.' (Cosine Score: 0.8439)




tweet: Computer scientists have apparently uncovered a covert server linking the Trump Organization to a Russian-based bank. pic.twitter.com/8f8n9xMzUU — Hillary Clinton (@HillaryClinton) November 1, 2016

Top 20 most similar vclaim in corpus:
Experts confirmed a computer server linke





tweet: I hope Obama serious about this hot food with ebt thing ?? — NickiM? (@Lilonikaa) October 19, 2016

Top 20 most similar vclaim in corpus:
Olive Garden restaurants are 'funding Trump's re-election in 2020.' (Cosine Score: 0.7197)




tweet: Once again hillary proves she can't handle classified information. Gives away America's nuclear response time to the World. What an idiot! — OneUSMarineRet (@MasterGuns1313) October 20, 2016

Top 20 most similar vclaim in corpus:
Major General Paul Eaton said President Trump's decertification of an existing nuclear deal with Iran deal 'dishonors America.' (Cosine Score: 0.8522)




tweet: When the president gives the order to launch a nuclear weapon, that’s it. The officer has to launch. It can take as little as four minutes. — Hillary Clinton (@HillaryClinton) October 20, 2016

Top 20 most similar vclaim in corpus:
Hillary Clinton revealed a classified response window of "four minutes" for a U.S. president to launch nuclear weapons. (Cosi





tweet: ?‼️?? #USA, #NewYork: Welfare Scammers - Muslims caught buying Soda with Food Stamps to sell at their store. More: https://t.co/74O8qmkW3npic.twitter.com/YfqYBpxIJJ— Onlinemagazin (@OnlineMagazin) September 30, 2016

Top 20 most similar vclaim in corpus:
A video depicts a Muslim woman fraudulently using food stamps to purchase soda to resell at her store. (Cosine Score: 0.8225)




tweet: UN Demands U.S. Pay Reparations To Blacks... https://t.co/qDMzdmzx88— DRUDGE REPORT (@DRUDGE_REPORT) September 28, 2016

Top 20 most similar vclaim in corpus:
A resolution passed in March 2016 forces residents of a Texas county to pay reparations to black citizens. (Cosine Score: 0.7976)




tweet: Outrageous! The UN Has Decided The U.S. Owes Reparations To Black People For A History Of 'Racial Terrorism'  https://t.co/ZAOj0fGrJw— RedFlagNews.com (@redflagnews) September 28, 2016

Top 20 most similar vclaim in corpus:
Miller County Judge Roy John McNatt wrote a racist memo about black peopl





tweet: During a recent interview with Australian journalist John Pilger, Assange said that Bernie Sanders was forced to... https://t.co/rxYa2FZIZ8 — Jonathan Henderson (@Dagan81) September 9, 2016

Top 20 most similar vclaim in corpus:
Matt Damon announced in March 2018 that he was moving his family to 'a safe place' in Australia because he was fed up with President Donald Trump and his policies. (Cosine Score: 0.6926)




tweet: #BreakingNews: We’re launching an exciting new savoury #condom range – Eggplant flavour! ? #CondomEmojipic.twitter.com/idA07EaiXC — Durex Global (@durex) September 5, 2016

Top 20 most similar vclaim in corpus:
The "Tide pod challenge" is a real viral phenomenon whereby people bite into Tide brand laundry detergent pods. (Cosine Score: 0.7139)




tweet: Phyllis Schlafly said “We’ll have a woman President over my dead body.” God accepts the deal. https://t.co/5lDfLA23Qq — Randi Rhodes (@RandiRhodes) September 6, 2016

Top 20 most similar vclaim in corpus:


In [44]:
df_t.loc[df_t['tweet_content'] == 'NYPD just came to my house bc Rutgers Police told them i’m a threat based on political statements i’ve made on campus and on twitter. — Kevin Allred (@KevinAllred) November 16, 2016','tweet_id'].item()

896